used the following as a base of how to process features:

https://www.kaggle.com/klyusba/house-prices-advanced-regression-techniques/lasso-model-for-regression-problem

In [233]:
# import libraries

import pandas as pd
import numpy as np
import math

In [234]:
# read in test and training data

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [235]:
# combine all data for processing

all_data = pd.concat( (train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:, 'MSSubClass':'SaleCondition']), 
                    ignore_index=True)

Handle missing values

In [236]:
# lot frontage - assume if absent, is sqrt or lot area

all_data.loc[all_data['LotFrontage']
             .isnull(), 'LotFrontage'] = np.sqrt(all_data.loc[
        all_data['LotFrontage'].isnull(), 'LotArea'])

Many features where NaN can be considered as an absence of that feature.

For other features, using the median might be more appropriate

In [237]:
# alley
all_data.loc[all_data.Alley.isnull(), 'Alley'] = 'NoAlley'

# masonry vaneer
all_data.loc[all_data.MasVnrType.isnull(), 'MasVnrType'] = 'NoVnr'
all_data.loc[all_data.MasVnrType=='NoVnr', 'MasVnrArea'] = 0

# basement quality, condition, exposure, finish etc.
all_data.loc[all_data.BsmtQual.isnull(), 'BsmtQual'] = 'NoBsmt'
all_data.loc[all_data.BsmtCond.isnull(), 'BsmtCond'] = 'NoBsmt'
all_data.loc[all_data.BsmtExposure.isnull(), 'BsmtExposure'] = 'NoBsmt'
all_data.loc[all_data.BsmtFinType1.isnull(), 'BsmtFinType1'] = 'NoBsmt'
all_data.loc[all_data.BsmtFinType2.isnull(), 'BsmtFinType2'] = 'NoBsmt'
all_data.loc[all_data.BsmtFinType1=='NoBsmt', 'BsmtFinSF1'] = 0
all_data.loc[all_data.BsmtFinType2=='NoBsmt', 'BsmtFinSF2'] = 0
all_data.loc[all_data.BsmtFinSF1.isnull(), 'BsmtFinSF1'] = all_data.BsmtFinSF1.median()
all_data.loc[all_data.BsmtQual=='NoBsmt', 'BsmtUnfSF'] = 0
all_data.loc[all_data.BsmtUnfSF.isnull(), 'BsmtUnfSF'] = all_data.BsmtUnfSF.median()
all_data.loc[all_data.BsmtQual=='NoBsmt', 'TotalBsmtSF'] = 0

# fireplace
all_data.loc[all_data.FireplaceQu.isnull(), 'FireplaceQu'] = 'NoFireplace'

# Garage finish, quality and condition
all_data.loc[all_data.GarageType.isnull(), 'GarageType'] = 'NoGarage'
all_data.loc[all_data.GarageFinish.isnull(), 'GarageFinish'] = 'NoGarage'
all_data.loc[all_data.GarageQual.isnull(), 'GarageQual'] = 'NoGarage'
all_data.loc[all_data.GarageCond.isnull(), 'GarageCond'] = 'NoGarage'
# also one entry has garage Area and Cars null - use typical value for this garage type
all_data.loc[all_data['GarageArea'].isnull(), 'GarageArea'] = all_data.loc[all_data['GarageType']=='Detchd', 'GarageArea'].mean()
all_data.loc[all_data['GarageCars'].isnull(), 'GarageCars'] = all_data.loc[all_data['GarageType']=='Detchd', 'GarageCars'].median()

# bathrooms
all_data.loc[all_data.BsmtFullBath.isnull(), 'BsmtFullBath'] = 0
all_data.loc[all_data.BsmtHalfBath.isnull(), 'BsmtHalfBath'] = 0

# Zoning. Assume if null, is most common type
all_data.loc[all_data.MSZoning.isnull(), 'MSZoning'] = 'RL'

# Utilities
all_data.loc[all_data.Utilities.isnull(), 'Utilities'] = 'AllPub'
all_data.loc[all_data['Electrical'].isnull(), 'Electrical'] = 'SBrkr'

# exterior
all_data.loc[all_data.Exterior1st.isnull(), 'Exterior1st'] = 'VinylSd'
all_data.loc[all_data.Exterior2nd.isnull(), 'Exterior2nd'] = 'VinylSd'
all_data.loc[all_data.Functional.isnull(), 'Functional'] = 'Typ'

# sale condition
all_data.loc[all_data.SaleCondition.isnull(), 'SaleCondition'] = 'Normal'
all_data.loc[all_data.SaleCondition.isnull(), 'SaleType'] = 'WD'

# pool quality
all_data.loc[all_data['PoolQC'].isnull(), 'PoolQC'] = 'NoPool'

# fence
all_data.loc[all_data['Fence'].isnull(), 'Fence'] = 'NoFence'

# other feartures
all_data.loc[all_data['MiscFeature'].isnull(), 'MiscFeature'] = 'None'


Quality based features, or features where order is clear, make numeric

In [238]:
all_data = all_data.replace(
    {'Utilities':{
            'AllPub': 1,
            'NoSeWa': 0,
            'NoSewr': 0,
            'ELO': 0
        },
    'Street':{
            'Pave': 1,
            'Grvl': 0
        },
     'FireplaceQu': {
            'Ex': 5, 
            'Gd': 4,
            'TA': 3,
            'Fa': 2,
            'Po': 1,
            'NoFireplace': 0
        },
     'Fence': {
            'GdPrv': 2,
            'GdWo': 2,
            'MnPrv': 1,
            'MnWW': 1,
            'NoFence': 0
        },
     'ExterQual': {
            'Ex': 5, 
            'Gd': 4,
            'TA': 3,
            'Fa': 2,
            'Po': 1
        },
     'ExterCond': {
            'Ex': 5, 
            'Gd': 4,
            'TA': 3,
            'Fa': 2,
            'Po': 1
        },
     'BsmtQual': {
            'Ex': 5, 
            'Gd': 4,
            'TA': 3,
            'Fa': 2,
            'Po': 1,
            'NoBsmt': 0
        },
     'BsmtExposure': {
            'Ex': 5, 
            'Gd': 4,
            'TA': 3,
            'Fa': 2,
            'Po': 1,
            'NoBsmt': 0
        },
     'BsmtCond': {
            'Ex': 5, 
            'Gd': 4,
            'TA': 3,
            'Fa': 2,
            'Po': 1,
            'NoBsmt': 0
        },
     'GarageQual': {
            'Ex': 5, 
            'Gd': 4,
            'TA': 3,
            'Fa': 2,
            'Po': 1,
            'NoGarage': 0
        },
     'KitchenQual': {
            'Ex': 5, 
            'Gd': 4,
            'TA': 3,
            'Fa': 2,
            'Po': 1
        },
     'Functional': {
            'Typ': 0,
            'Min2': 1,
            'Min1': 1,
            'Mod': 2,
            'Maj1': 3,
            'Maj2': 4,
            'Sev': 5,
            'Sal': 6
        }
    }
)

In [239]:
# Heating QC

all_data = all_data.replace({ 'HeatingQC': {
        'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}
    })

In [240]:
# central air
all_data = all_data.replace({'CentralAir': {
            'Y': 1, 'N':0
        }})

# paved drive
all_data = all_data.replace({'PavedDrive': {
            'Y': 1, 'P': 0, 'N': 0
        }})

In [241]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 79 columns):
MSSubClass       2919 non-null int64
MSZoning         2919 non-null object
LotFrontage      2919 non-null float64
LotArea          2919 non-null int64
Street           2919 non-null int64
Alley            2919 non-null object
LotShape         2919 non-null object
LandContour      2919 non-null object
Utilities        2919 non-null int64
LotConfig        2919 non-null object
LandSlope        2919 non-null object
Neighborhood     2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
BldgType         2919 non-null object
HouseStyle       2919 non-null object
OverallQual      2919 non-null int64
OverallCond      2919 non-null int64
YearBuilt        2919 non-null int64
YearRemodAdd     2919 non-null int64
RoofStyle        2919 non-null object
RoofMatl         2919 non-null object
Exterior1st      2919 non-null object
Exterior2nd      2919 non-

In [242]:
# good neighborhoods: 'NridgHt', 'Crawfor', 'StoneBr',
# 'Somerst', 'NoRidge'

all_data = all_data.replace({'Neighborhood': {
           'Blmngtn':0, 'Blueste':0, 'BrDale':0,
            'BrkSide':0, 'ClearCr':0, 'CollgCr':0,
            'Crawfor':1, 'Edwards':0, 'Gilbert':0,
            'IDOTRR': 0, 'MeadowV':0, 'Mitchel':0,
            'Names':0, 'NoRidge':1, 'NPkVill':0,
            'NridgHt':1, 'NWAmes':0, 'OldTown':0,
            'SWISU':0, 'Sawyer':0, 'SawyerW':0,
            'Somerst':1, 'StoneBr':1, 'Timber':0,
            'Veenker':0 
        }})

In [243]:
all_data = all_data.replace({'MasVnrType': {
            'BrkCmn': 1,
            'BrkFace': 1,
            'CBlock': 1,
            'Stone': 1,
            'None': 0
        }})

In [244]:
# Drop:
# MSZoning, Alley, LandContour, LandSlope, Condition1,
# Condition2, BldgType, HouseStyle, RoofStyle, 
# RoofMatl, Exterior1st, Exterior2nd, Foundation,
# BsmtExposure, BsmtFinType1, BsmtFinType2, Heating, 
# Electrical, GarageType, GarageFinish, GarageCond,
# PoolQC, SaleType, SaleCondition


reshape data and prep for model!

In [245]:
X = all_data.drop('MSZoning', axis=1)

to_drop = ['Alley', 'LandContour', 'LandSlope', 
           'Condition1', 'Condition2', 'BldgType', 
           'HouseStyle', 'RoofStyle', 'RoofMatl', 
           'Exterior1st', 'Exterior2nd', 'Foundation',
           'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 
           'Heating', 'Electrical', 'GarageType', 
           'GarageFinish', 'GarageCond', 'PoolQC', 
           'SaleType', 'SaleCondition', 'GarageYrBlt',
           'LotShape', 'LotConfig', 'Neighborhood', 
           'MasVnrType', 'Fence', 'MiscFeature', 'MasVnrArea']
X.drop(to_drop, axis=1, inplace=True)


In [276]:
X_train = X[:train.shape[0]]
X_test = X[train.shape[0]:]

In [277]:
X_test.head()

,MSSubClass,LotFrontage,LotArea,Street,Utilities,OverallQual,OverallCond,YearBuilt,YearRemodAdd,ExterQual,...,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
1460,20,80.0,11622,1,1,5,6,1961,1961,3,...,1,140,0,0,0,120,0,0,6,2010
1461,20,81.0,14267,1,1,6,6,1958,1958,3,...,1,393,36,0,0,0,0,12500,6,2010
1462,60,74.0,13830,1,1,5,5,1997,1998,3,...,1,212,34,0,0,0,0,0,3,2010
1463,60,78.0,9978,1,1,6,6,1998,1998,3,...,1,360,36,0,0,0,0,0,6,2010
1464,120,43.0,5005,1,1,8,5,1992,1992,4,...,1,0,82,0,0,144,0,0,1,2010


In [247]:
y = train.SalePrice

In [248]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.cross_validation import KFold
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.cross_validation import cross_val_score

In [300]:
X_train.shape

(1460, 47)

In [301]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2, interaction_only=True)
poly_features = poly.fit_transform(X_train)
poly_df = pd.DataFrame(poly_features)
X_train = pd.concat([X_train, poly_df], axis=1)
X_train.shape

(1460, 1176)

In [251]:
forest = RandomForestRegressor(max_features=len(X_train.columns))

parameter_grid = {
    'max_depth' : [None,1,3,9],
    'min_samples_split' : [1,2,3],
    'n_estimators' : [3,10,30,100,200,400]
}

cross_validation = KFold(len(y),n_folds=5)

grid_search = GridSearchCV(
    forest,
    param_grid=parameter_grid,
    cv=cross_validation
)

grid_search.fit(X_train, y)

print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

print(grid_search.score(X_train, y))

Best score: 0.86212753393
Best parameters: {'min_samples_split': 3, 'n_estimators': 30, 'max_depth': None}
0.977572238659


In [278]:
X_test.KitchenQual = X_test.KitchenQual.fillna(all_data.KitchenQual.median())

,MSSubClass,LotFrontage,LotArea,Street,Utilities,OverallQual,OverallCond,YearBuilt,YearRemodAdd,ExterQual,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.00000,...,1460.000000,1460.000000,1460.000000,1460.000000,1.460000e+03,1460.000000,1.460000e+03,1460.000000,1.460000e+03,1460.000000
mean,56.897260,76.644196,10516.828082,0.995890,0.999315,6.099315,5.575342,1971.267808,1984.865753,3.39589,...,156.410959,1538.082192,98.711644,30240.421918,7.109589e+02,13.779452,5.536188e+03,266.228767,8.732121e+04,12692.723288
std,42.300571,31.274670,9981.264932,0.063996,0.026171,1.382997,1.112799,30.202904,20.645407,0.57428,...,5976.454348,30493.447904,396.722423,111948.229564,2.716570e+04,237.271645,8.061999e+04,2689.105244,9.958443e+05,5427.104981
min,20.000000,21.000000,1300.000000,0.000000,0.000000,1.000000,1.000000,1872.000000,1950.000000,2.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,2006.000000
25%,20.000000,60.000000,7553.500000,1.000000,1.000000,5.000000,5.000000,1954.000000,1967.000000,3.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,10030.000000
50%,50.000000,73.000000,9478.500000,1.000000,1.000000,6.000000,5.000000,1973.000000,1994.000000,3.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,12054.000000
75%,70.000000,90.000000,11601.500000,1.000000,1.000000,7.000000,6.000000,2000.000000,2004.000000,4.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,16056.000000
max,190.000000,405.783193,215245.000000,1.000000,1.000000,10.000000,9.000000,2010.000000,2010.000000,5.00000,...,228360.000000,880000.000000,3492.000000,963840.000000,1.038000e+06,5904.000000,1.480428e+06,66400.000000,3.110850e+07,24108.000000


In [302]:
poly = PolynomialFeatures(2, interaction_only=True)
poly_features = poly.fit_transform(X_test)
poly_df = pd.DataFrame(poly_features)

In [313]:
testy = X_test.reset_index()
testy.drop('index', axis=1, inplace=True)
testy = pd.concat([testy, poly_df], axis=1)
print(poly_df.shape)
print(X_test.shape)
print(testy.shape)


(1459, 1129)
(1459, 47)
(1459, 1176)


In [314]:
output = grid_search.predict(testy)
df_output = pd.DataFrame()
df_output['Id'] = test['Id']
df_output['SalePrice'] = output
df_output[['Id', 'SalePrice']].to_csv('results2-v3.csv', index=False)